We are going to start creating a series for training.

y[0] = x[0]

y[1] = x[1]

y[2] = x[0]+x[1]


We are going to create different trainning sequences with different startup variables

In [63]:
import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn

In [51]:
num_seqs=7
seq_len=15
seqs_x=np.array([]).reshape(-1,seq_len)
seqs_y=np.array([]).reshape(-1,seq_len)
for i in xrange(num_seqs):
    x=np.empty((0),dtype=np.float64)
    for j in xrange(seq_len):
        x=np.append(x,np.float64(np.random.randint(0,50)))

    y=np.empty_like(x)
    y[0]=x[0]
    y[1]=x[1]
    for j in xrange(2,len(x)):
        y[j]=x[j-1]+x[j-2]
    seqs_x=np.concatenate((seqs_x, np.reshape(x,(1,-1))))
    seqs_y=np.concatenate((seqs_y, np.reshape(y,(1,-1))))
    
    

print "Sequences X:"
print seqs_x
print "Sequences Y:"
print seqs_y

Sequences X:
[[23. 20. 32. 32.  3. 14. 22. 28.  5. 22.  4. 27. 26. 33. 29.]
 [ 3. 33.  1. 47. 48. 14. 13. 25.  5. 25. 19. 41. 10. 45. 25.]
 [24. 11. 13. 30.  7. 13.  7.  2.  0.  1. 19. 38.  6. 31. 12.]
 [ 0. 23. 49.  8. 33. 28.  0. 46. 10. 38. 29. 15.  8.  2. 19.]
 [29. 44. 30. 17. 28.  1. 39.  9. 43. 47.  5. 37. 47. 45. 11.]
 [19. 33. 29. 49.  3. 35. 13. 13. 11. 12. 35. 22. 47. 25. 21.]
 [47. 14. 25. 40. 21. 19. 29. 41.  3. 27. 34. 31. 46. 48. 43.]]
Sequences Y:
[[23. 20. 43. 52. 64. 35. 17. 36. 50. 33. 27. 26. 31. 53. 59.]
 [ 3. 33. 36. 34. 48. 95. 62. 27. 38. 30. 30. 44. 60. 51. 55.]
 [24. 11. 35. 24. 43. 37. 20. 20.  9.  2.  1. 20. 57. 44. 37.]
 [ 0. 23. 23. 72. 57. 41. 61. 28. 46. 56. 48. 67. 44. 23. 10.]
 [29. 44. 73. 74. 47. 45. 29. 40. 48. 52. 90. 52. 42. 84. 92.]
 [19. 33. 52. 62. 78. 52. 38. 48. 26. 24. 23. 47. 57. 69. 72.]
 [47. 14. 61. 39. 65. 61. 40. 48. 70. 44. 30. 61. 65. 77. 94.]]


In [55]:
th = np.int(num_seqs*0.7)
train_x= seqs_x[:th]
test_x = seqs_x[th:]
train_y= seqs_y[:th]
test_y = seqs_y[th:]

In [61]:
lstm_size = 10
num_features = 1

In [117]:
tf.reset_default_graph()

#The input data is going to be [batch_size]
x_data = tf.placeholder(tf.float32, [None, 1],"x_data")

#The output data is going to be [batch_size, 1]
y_real = tf.placeholder(tf.float32, [None, 1],"y_real")

lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size, state_is_tuple=True,name="LSTM_cell")
c_in=tf.placeholder(tf.float32, [None, lstm_size],name="c_in")
h_in=tf.placeholder(tf.float32, [None, lstm_size],name="h_in")
                    
init_state = tf.contrib.rnn.LSTMStateTuple( c=c_in, h=h_in)


# The value of state is updated after processing each batch of words.
output_rnn, out_state = lstm(x_data, init_state)

W_out = tf.get_variable("Wout",initializer=tf.random_normal([lstm_size,1]))
b_out = tf.get_variable("b_out",initializer=tf.random_normal([1]))

# The LSTM output can be used to make next word predictions
out = tf.matmul(output_rnn, W_out) + b_out

loss = tf.losses.mean_squared_error(y_real,out)
train_op = tf.train.AdamOptimizer().minimize(loss)

sess=tf.Session()
sess.run(tf.global_variables_initializer())

# We save the graph to analyze it in Tensorboard
train_writer = tf.summary.FileWriter('./graphtf',sess.graph)

Le'ts try with an example, yust one sample from the list

In [118]:
np_c = np.zeros((1, lstm_size),np.float32)
np_h = np.zeros((1, lstm_size),np.float32)

o = sess.run([out], feed_dict={x_data: train_x[0:1,0].reshape(-1,1), 
                                      y_real:train_y[0:1,0].reshape(-1,1),
                                      c_in:np_c, h_in:np_h})

print "The estimated output without training is:"
print np.squeeze(o)
print "But it should be:"
print train_y[1,0]

The estimated output without training is:
0.0042195916
But it should be:
3.0


Let's train the model

In [119]:
max_epocs=10000
for j in xrange(max_epocs):
    np_c = np.zeros((train_x.shape[0], lstm_size),np.float32)
    np_h = np.zeros((train_x.shape[0], lstm_size),np.float32)
    for i in xrange(train_x.shape[1]):        
        new_state,myloss,_ = sess.run([out_state,loss,train_op], 
               feed_dict={x_data: train_x[:,i].reshape(-1,1), 
                          y_real:train_y[:,i].reshape(-1,1),
                          c_in:np_c, h_in:np_h
                         })        
        np_c=new_state.c
        np_h=new_state.h
    if j%200 == 0:
        print myloss

2046.0165
438.22595
373.78998
337.576
340.35507
252.77315
175.91467
163.68973
126.21148
125.45929
115.56317
67.36999
48.805077
48.48239
51.104794
16.888826
3.3809328
29.782288
57.244755
19.60713
59.499363
29.204899
11.019059
8.672547
9.082928
3.394328
1.7342178
0.6894849
23.618866
18.484379
31.286089
86.694435
80.29189
9.22579
29.815407
65.00361
7.24303
28.543066
29.233694
107.85974
182.72688
86.162254
213.87238
81.63963
59.246887
13.777568
34.940994
46.585064
22.325748
15.47254


Let's check with the known sequences

In [120]:
np_c = np.zeros((1, lstm_size),np.float32)
np_h = np.zeros((1, lstm_size),np.float32)

seq_id=1

out_seq=np.array([])
for i in xrange(test_x.shape[1]):
    out_val,new_state = sess.run([out,out_state], feed_dict={x_data: test_x[seq_id,i].reshape(-1,1),       
                                                             c_in:np_c, h_in:np_h})
    out_seq = np.append(out_seq,out_val)
    np_c=new_state.c
    np_h=new_state.h

print "The estimated output sequence is:"
print out_seq
print "The real output sequence is:"
print test_y[seq_id,:]

The estimated output sequence is:
[19.21244049 31.39744949 38.73437881 31.38534737 54.78489685 11.93888664
 57.93574142 51.94184494 34.24225235 19.72789574 34.66477585 51.43584824
 50.71542358 55.28791809 72.36883545]
The real output sequence is:
[19. 33. 52. 62. 78. 52. 38. 48. 26. 24. 23. 47. 57. 69. 72.]


## Sequences with N-dimensions

We are going to start creating a series for training.
\\[
  y[0] = x[0]
  y[1] = x[1]
  y[2] = x[0]+x[1]    
\\]



In [ ]:
num_seqs=7
seq_len=15
seqs_x=np.array([]).reshape(-1,seq_len)
seqs_y=np.array([]).reshape(-1,seq_len)
for i in xrange(num_seqs):
    x=np.empty((0),dtype=np.float64)
    for j in xrange(seq_len):
        x=np.append(x,np.float64(np.random.randint(0,50)))

    y=np.empty_like(x)
    y[0]=x[0]
    y[1]=x[1]
    for j in xrange(2,len(x)):
        y[j]=x[j-1]+x[j-2]
    seqs_x=np.concatenate((seqs_x, np.reshape(x,(1,-1))))
    seqs_y=np.concatenate((seqs_y, np.reshape(y,(1,-1))))
    
    

print "Sequences X:"
print seqs_x
print "Sequences Y:"
print seqs_y


In [109]:
test_y[seq_id,:]

array([29., 44., 73., 74., 47., 45., 29., 40., 48., 52., 90., 52., 42.,
       84., 92.])

In [17]:
train_x = np.array([[[1], [2], [5], [6]],
          [[5], [7], [7], [8]],
          [[3], [4], [5], [7]]])

train_x = np.array([[[1], [2], [5], [6]],
          [[5], [7], [7], [8]],
          [[3], [4], [5], [7]],
          [[2], [3], [7], [8]]])

train_y = np.array([[1, 3, 7, 11],
           [5, 12, 14, 15],
           [3, 7, 9, 12],
           [2, 5, 10 ,15]])

test_x = np.array([[[1], [2], [3], [4]],
          [[4], [5], [6], [7]]])

In [3]:
time_steps = 4
num_features = 1
lstm_size=10

In [18]:
tf.reset_default_graph()


batch_step = tf.placeholder(tf.float32, [None, num_features],"x_train")
y_real = tf.placeholder(tf.float32, [None, num_features])

batch_size = tf.shape(batch_step)[0]
lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size, state_is_tuple=True,name="LSTM_cell")
# Initial state of the LSTM memory.
#init_state = state = lstm.zero_state(batch_size,tf.float32)
#c_in=tf.zeros((batch_size, lstm_size), np.float32,name="c_in")
#h_in=tf.zeros((batch_size, lstm_size),np.float32,name="h_in")
c_in=tf.placeholder(tf.float32, [None, lstm_size],name="c_in")
h_in=tf.placeholder(tf.float32, [None, lstm_size],name="h_in")
                    
init_state = tf.contrib.rnn.LSTMStateTuple( c=c_in, h=h_in)
#reset_state=tf.identity_n(init_state)


loss = 0.0
# The value of state is updated after processing each batch of words.
output_rnn, out_state = lstm(batch_step, init_state)


W_out = tf.get_variable("Wout",initializer=tf.random_normal([lstm_size,1]))
b_out = tf.get_variable("b_out",initializer=tf.random_normal([1]))


# The LSTM output can be used to make next word predictions
out = tf.matmul(output_rnn, W_out) + b_out

loss = tf.losses.mean_squared_error(y_real,out)
train_op = tf.train.AdamOptimizer().minimize(loss)

sess=tf.Session()
sess.run(tf.global_variables_initializer())

np_c = np.zeros((train_x.shape[0], lstm_size),np.float32)
np_h = np.zeros((train_x.shape[0], lstm_size),np.float32)

print sess.run([out,loss,train_op], 
               feed_dict={batch_step: train_x[:,0,:], 
                          y_real:train_y[:,0].reshape(-1,1),
                          c_in:np_c, h_in:np_h
                         })
train_writer = tf.summary.FileWriter('./graphtf',sess.graph)


[array([[-0.950649  ],
       [-0.6088793 ],
       [-0.75499594],
       [-0.84744596]], dtype=float32), 14.368125, None]


In [21]:
max_epocs=1000
for j in xrange(max_epocs):
    np_c = np.zeros((train_x.shape[0], lstm_size),np.float32)
    np_h = np.zeros((train_x.shape[0], lstm_size),np.float32)
    for i in xrange(4):        
        new_state,myloss,_ = sess.run([out_state,loss,train_op], 
               feed_dict={batch_step: train_x[:,i,:], 
                          y_real:train_y[:,i].reshape(-1,1),
                          c_in:np_c, h_in:np_h
                         })        
        np_c=new_state.c
        np_h=new_state.h
    if j%100 == 0:
        print myloss

0.49185872
0.40784678
0.34643236
0.30182305
0.27053022
0.24956086
0.23522538
0.2237002
0.21229109
0.19987653


In [23]:
np_c = np.zeros((test_x.shape[0], lstm_size),np.float32)
np_h = np.zeros((test_x.shape[0], lstm_size),np.float32)

for i in xrange(4):
    out_val,new_state = sess.run([out,out_state], feed_dict={batch_step: test_x[:,i,:],c_in:np_c, h_in:np_h})
    np_c=new_state.c
    np_h=new_state.h
    print out_val
    

[[0.9709455]
 [4.0639324]]
[[2.9082494]
 [9.049345 ]]
[[ 4.6729946]
 [11.843623 ]]
[[ 6.1187854]
 [12.876646 ]]


In [428]:
test_x

array([[[1],
        [2],
        [3],
        [4]],

       [[4],
        [5],
        [6],
        [7]]])

In [ ]:
[[ 1.6971655  2.9445994  4.9434905  7.257107 ]
 [ 4.436798   9.061197  11.678692  12.575489 ]]


https://medium.com/@erikhallstrm/hello-world-rnn-83cd7105b767

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn

In [37]:
class SeriesPredictor:
    def __init__(self, input_dim, seq_size, hidden_dim=10):
        
        self.input_dim=input_dim
        self.seq_size=seq_size
        self.hidden_dim=hidden_dim
        
        self.W_out = tf.Variable(tf.random_normal([hidden_dim,1]),name='W_out')
        self.b_out = tf.Variable(tf.random_normal([1]),name='b_out')
        self.x = tf.placeholder(tf.float32, [None, seq_size, input_dim])
        self.y = tf.placeholder(tf.float32, [None, seq_size])
        
        
        self.cost = tf.reduce_mean(tf.square(self.model()-self.y))
        self.train_op = tf.train.AdamOptimizer().minimize(self.cost)
        
        self.saver = tf.train.Saver()
        
        
    def model(self):
        """
        :param x: inputs of size [T, batch_size, input_size]
        :param W: matrix of fully-connected output layer weights
        :param b: vector of fully-connected output layer biases
        """
        cell = rnn.BasicLSTMCell(self.hidden_dim)
        outputs, states = tf.nn.dynamic_rnn(cell, self.x, dtype=tf.float32)
        num_examples = tf.shape(self.x)[0]
        W_repeated = tf.tile(tf.expand_dims(self.W_out,0), [num_examples,1,1])
        out = tf.matmul(outputs, W_repeated)+self.b_out
        out = tf.squeeze(out)
        return out
    
    def train(self, train_x, train_y):
        with tf.Session() as sess:
            tf.get_variable_scope().reuse_variables()
            sess.run(tf.global_variables_initializer())
            for i in range(1000):
                _, mse = sess.run([self.train_op, self.cost], feed_dict={self.x: train_x, self.y: train_y})
                if i% 100 == 0:
                    print(i,mse)
            save_path = self.saver.save(sess,'./model.ckpt')
            print('Model saved to {}'.format(save_path))
                
        
    def test(self,test_x):
        with tf.Session() as sess:
            tf.get_variable_scope().reuse_variables()
            self.saver.restore(sess, './model.ckpt')
            output = sess.run(self.model(), feed_dict={self.x: test_x})
            print(output)

    
        

In [10]:
class SeriesPredictor:
    def __init__(self, input_dim, seq_size, hidden_dim=10):
        
        self.input_dim=input_dim
        self.seq_size=seq_size
        self.hidden_dim=hidden_dim
        
        self.W_out = tf.get_variable("Wout",initializer=tf.random_normal([hidden_dim,1]))
        self.b_out = tf.get_variable("b_out",initializer=tf.random_normal([1]))
        self.x = tf.placeholder(tf.float32, [None, seq_size, input_dim])
        self.y = tf.placeholder(tf.float32, [None, seq_size])
        
        self.mymodel = self.model()
        self.cost = tf.reduce_mean(tf.square(self.mymodel-self.y))
        self.train_op = tf.train.AdamOptimizer().minimize(self.cost)
        
        self.saver = tf.train.Saver()
        
    def model(self):
        """
        :param x: inputs of size [T, batch_size, input_size]
        :param W: matrix of fully-connected output layer weights
        :param b: vector of fully-connected output layer biases
        """
        cell = rnn.BasicLSTMCell(self.hidden_dim)
        outputs, states = tf.nn.dynamic_rnn(cell, self.x, dtype=tf.float32)
        num_examples = tf.shape(self.x)[0]
        W_repeated = tf.tile(tf.expand_dims(self.W_out,0), [num_examples,1,1])
        out = tf.matmul(outputs, W_repeated)+self.b_out
        out = tf.squeeze(out)
        return out
    
    def train(self, train_x, train_y):
        sess = tf.Session()
        sess.run(tf.global_variables_initializer())
        for i in range(1000):
            _, mse = sess.run([self.train_op, self.cost], feed_dict={self.x: train_x, self.y: train_y})
            if i% 100 == 0:
                print(i,mse)
        return sess
                
        
    def test(self,sess,test_x):        
        output = sess.run(self.mymodel, feed_dict={self.x: test_x})
        print(output)

    
        

In [11]:
tf.reset_default_graph()
predictor = SeriesPredictor(input_dim=1, seq_size=4, hidden_dim=10)
train_x = [[[1], [2], [5], [6]],
          [[5], [7], [7], [8]],
          [[3], [4], [5], [7]]]
train_y = [[1, 3, 7, 11],
           [5, 12, 14, 15],
           [3, 7, 9, 12]]
sess=predictor.train(train_x, train_y)

test_x = [[[1], [2], [3], [4]],
          [[4], [5], [6], [7]]]

predictor.test(sess,test_x)

(0, 104.54908)
(100, 49.596928)
(200, 13.441304)
(300, 4.84034)
(400, 3.223334)
(500, 2.582983)
(600, 1.9891154)
(700, 1.4041783)
(800, 1.0117636)
(900, 0.73919386)
[[ 0.69609344  2.6687462   5.052478    6.9160995 ]
 [ 4.0849714   9.2390785  11.849617   12.85392   ]]


In [17]:
train_x[0][0]

[[1], [2], [5], [6]]